In [22]:
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv('../05_data_scraping/.env')
import os
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
from dbio import er_to_db, db_connect
from datetime import datetime

In [23]:
def string_delete(text) :
    return text.replace("<b>","").replace("</b>", "").replace("&quot;", "")

def parse_date(date_str):
    # 날짜 문자열을 datetime 객체로 변환
    return datetime.strptime(date_str, "%a, %d %b %Y %H:%M:%S %z")

def make_df(df_name) :
    for i in df_name :
        for idx in range(0, len(df_name[i])) : 
            df_name[i][idx] = string_delete(df_name[i][idx])
    df_name['pubDate'] = df_name['pubDate'].apply(parse_date)
    return df_name

In [26]:
client_id = os.getenv('client_id') # 환경 변수에 접근
client_pw = os.getenv('client_pw') # 환경 변수에 접근

url = "https://openapi.naver.com/v1/search/news.json"
payload = {'query' : '건강에 좋은 음식', 'display' : 10, 'start' : 1, 'sort' : 'date'}
headers = {'X-Naver-Client-Id' : client_id, 'X-Naver-Client-Secret' : client_pw}
r = requests.get(url, params = payload, headers = headers)
if r.status_code == 200 :
    data = r.json()
#     print(data)
else :
    print("Error Code:", r.status_code)
    

df = pd.json_normalize(data['items'])
make_df(df)

C:\Users\5-08\AppData\Local\Temp\ipykernel_15464\3190665787.py:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_name[i][idx] = string_delete(df_name[i][idx])


,title,originallink,link,description,pubDate
0,아침만 먹어도 이런 효과가…아무리 바빠도 아침 식사를 챙겨야 하는 이...,https://www.wikitree.co.kr/articles/993687,https://www.wikitree.co.kr/articles/993687,하지만 아침 식사를 간단하게라도 챙겨 먹는 편이 건강을 지키는 데에 좋다는 사실은 ...,2024-10-23 11:32:00+09:00
1,“5일 만에 4kg 감량”… 다이어트에 좋다고 틱톡서 난리 난 ‘올챙이 물...,https://health.chosun.com/site/data/html_dir/2...,https://n.news.naver.com/mnews/article/346/000...,최근 영국의 건강 관련 뉴스나 트렌드를 분석하는 출판사 ‘더 컨벌세이션(The co...,2024-10-23 11:31:00+09:00
2,[유통가 레이더] '배추 대란'에 대처하는 이마트의 전략... 절임배추 사...,https://www.greened.kr/news/articleView.html?i...,https://www.greened.kr/news/articleView.html?i...,"만드는 좋은 기회가 됐다. 향후, 우수한 기업들로부터 양질의 원료를 공급받아 제품의...",2024-10-23 10:42:00+09:00
3,[안녕한 부산] 특색 있는 노인일자리 '어묵168'...매장에 오면 즐겁습니...,https://news.bbsi.co.kr/news/articleView.html?...,https://news.bbsi.co.kr/news/articleView.html?...,"단순히 노인 일자리는 좋은 사업이다라는 막연한 의미보다는 가장 느린 어묵, 가장 맛...",2024-10-23 10:40:00+09:00
4,스마트폰 속 건강지키미…달라진 '삼성 헬스' 뜯어보니,https://www.seoultimes.news/news/article.html?...,https://www.seoultimes.news/news/article.html?...,well Connected Health)’와 파트너십을 바탕으로 건강 기록을 제공한...,2024-10-23 10:34:00+09:00
5,"환절기, 다리 붓고 어지럽다면 ‘심장’을 의심하라",https://www.hani.co.kr/arti/society/health/116...,https://n.news.naver.com/mnews/article/028/000...,저는 너무 좋은 밥맛과 너무 정확한 배꼽시계(?) 덕분에 1~2주 사이에 체중이 3...,2024-10-23 10:32:00+09:00
6,[관절염 명의의 진료실] 현미의 피틴산 논란 단점이 아니라 장점인 이유,http://www.ikunkang.com/news/articleView.html?...,http://www.ikunkang.com/news/articleView.html?...,"할 음식이라 주장했었다. 하지만 현미나 콩을 많이 섭취할수록 당뇨병, 심장병, 암 ...",2024-10-23 10:30:00+09:00
7,[오늘 뭐 먹지] 스타벅스·신세계L&amp;B·아워홈 외,http://www.newsprime.co.kr/news/article.html?n...,http://www.newsprime.co.kr/news/article.html?n...,다양한 음식과 페어링하기 좋다. '벨라스 가든'은 세계적인 와인 전문지 'Wine ...,2024-10-23 10:18:00+09:00
8,[아미랑] 속 더부룩할 때 탄산음료 찾으시나요? 건강 위해 ‘이 음료’...,https://health.chosun.com/site/data/html_dir/2...,https://n.news.naver.com/mnews/article/346/000...,오늘은 탄산음료의 위험성과 암 환자가 마시기 좋은 음료에 대해 알려드리겠습니다. 단...,2024-10-23 10:06:00+09:00
9,PURSUIT OF BLUE / 김영대,https://www.allurekorea.com/2024/10/23/pursuit...,https://www.allurekorea.com/2024/10/23/pursuit...,"아직도 저 자신에게 질 때가 많은데, 좋은 것을 먹고 아름다운 걸 보면 좀 위로받는...",2024-10-23 10:02:00+09:00


In [29]:
df.head(2)

,title,originallink,link,description,pubDate
0,아침만 먹어도 이런 효과가…아무리 바빠도 아침 식사를 챙겨야 하는 이...,https://www.wikitree.co.kr/articles/993687,https://www.wikitree.co.kr/articles/993687,하지만 아침 식사를 간단하게라도 챙겨 먹는 편이 건강을 지키는 데에 좋다는 사실은 ...,2024-10-23 11:32:00+09:00
1,“5일 만에 4kg 감량”… 다이어트에 좋다고 틱톡서 난리 난 ‘올챙이 물...,https://health.chosun.com/site/data/html_dir/2...,https://n.news.naver.com/mnews/article/346/000...,최근 영국의 건강 관련 뉴스나 트렌드를 분석하는 출판사 ‘더 컨벌세이션(The co...,2024-10-23 11:31:00+09:00


In [30]:
df = df.drop(['originallink', 'link'], axis = 1)
df.head()

,title,description,pubDate
0,아침만 먹어도 이런 효과가…아무리 바빠도 아침 식사를 챙겨야 하는 이...,하지만 아침 식사를 간단하게라도 챙겨 먹는 편이 건강을 지키는 데에 좋다는 사실은 ...,2024-10-23 11:32:00+09:00
1,“5일 만에 4kg 감량”… 다이어트에 좋다고 틱톡서 난리 난 ‘올챙이 물...,최근 영국의 건강 관련 뉴스나 트렌드를 분석하는 출판사 ‘더 컨벌세이션(The co...,2024-10-23 11:31:00+09:00
2,[유통가 레이더] '배추 대란'에 대처하는 이마트의 전략... 절임배추 사...,"만드는 좋은 기회가 됐다. 향후, 우수한 기업들로부터 양질의 원료를 공급받아 제품의...",2024-10-23 10:42:00+09:00
3,[안녕한 부산] 특색 있는 노인일자리 '어묵168'...매장에 오면 즐겁습니...,"단순히 노인 일자리는 좋은 사업이다라는 막연한 의미보다는 가장 느린 어묵, 가장 맛...",2024-10-23 10:40:00+09:00
4,스마트폰 속 건강지키미…달라진 '삼성 헬스' 뜯어보니,well Connected Health)’와 파트너십을 바탕으로 건강 기록을 제공한...,2024-10-23 10:34:00+09:00


In [31]:
df['description'][0]

'하지만 아침 식사를 간단하게라도 챙겨 먹는 편이 건강을 지키는 데에 좋다는 사실은 널리 알려져 있는 상식이기도 하다. 아침 식사의 중요성과 아침에 먹기 좋은 음식들에 대해 알아보자. 아침 식사는 과식과 폭식을 막아... '